### Задание по программированию: Логистическая регрессия

In [213]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import roc_auc_score

#### 1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [214]:
data = pd.read_csv("./data-logistic.csv", header = None)

#### 2. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [193]:
def f(X, Y, W, k, c, idx):
    l = len(X)
    weight = 0.0
    for i in range(l):
        weight += Y[i] * X.iloc[i, idx] * (1.0 - (1.0 / (1.0 + math.exp(-Y[i] * np.dot(X.iloc[i], W)))))
    return W[idx] + weight * k / l - k * c * W[idx]

def new_weights(X, Y, W, k, c):
    return np.array([f(X, Y, W, k, c, i) for i in range(len(W))])

def gradient_descent(X, Y, W, k, c, n = 10000, precision = 1e-5):
    prev_weights = np.array(W)
    for i in range(n):
        n_weights = new_weights(X, Y, prev_weights, k, c)
        distance = np.linalg.norm(n_weights - prev_weights)
        prev_weights = n_weights
        if float(distance) <= precision:
            break
    return prev_weights
        

In [215]:
X = data.iloc[:, 1:3]
y = data.iloc[:, 0]

k = 0.1 # gradient step
w_init = [0, 0] # initial weights

#### 3. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [206]:
w = gradient_descent(X, y, w_init, k, 0)

In [216]:
w_reg = gradient_descent(X, Y, w_init, k, 10)

#### 4. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. 
#### На вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. 

##### Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [202]:
def sigm(X, W):
    return 1.0 / (1 + math.exp(-W[0] * X[1] - W[1] * X[2]))

In [204]:
y_score = X.apply(lambda x : sigm(x, w), axis = 1)
y_reg_score = X.apply(lambda x : sigm(x, w_reg), axis = 1)

#### В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

In [217]:
auc = roc_auc_score(y, y_score)
reg_auc = roc_auc_score(y, y_reg_score)

#### Значение AUC-ROC без регуляризации

In [218]:
auc

0.9268571428571428

#### Значение AUC-ROC с использованием L2 регуляризации

In [210]:
reg_auc

0.9362857142857142